In [60]:
import pandas as pd
import numpy as np
import regex as re
import matplotlib.pyplot as plt
import glob, os
import time
from datetime import datetime
import spacy

In [61]:
df = pd.read_excel(r'./DatabaseTableUpload/Appendix_A_Capstone_DataSharingProposal.xlsx', sheet_name='A.25_ServiceNow_Incidents') # provide path for single .csv file
# directory = r'C:\Users\RJ\Downloads\2021_1021_re_esg\\' # or provide a directory that contains a collection of .csv/.xlsx files:

start_time = datetime.now()

In [62]:
# source_type = r'csv' # can be: directory, csv, or xlsx
# if source_type == 'directory':
#     df = pd.concat(map(lambda file: pd.read_csv(file, sep= ";", encoding = "utf-8-sig", low_memory=False), \
#                        glob.glob(os.path.join('', directory + '*.csv'))))
# elif source_type == r'csv':
#     df = pd.read_csv(csv, sep=';', low_memory=False)
# elif source_type == r'xlsx':
#     sheet_name = 1
#     df = pd.read_excel(path, sheet_name=sheet_name)

In [63]:
output_dir = r'./PII'

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Columns: 118 entries, parent to category
dtypes: float64(72), object(46)
memory usage: 12.1+ KB


In [65]:
for col in df.columns:
    print(col)

parent
caused_by
watch_list
u_browser_type
u_swap
upon_reject
sys_updated_on
origin_table
skills
u_gl_code
number
u_moniker
state
knowledge
order
contract
impact
active
priority
sys_domain_path
u_asset_substatus
business_duration
group_list
approval_set
u_public_pc
needs_attention
universal_request
short_description
correlation_display
u_error_code
work_start
additional_assignee_list
u_requestor
notify
service_offering
sys_class_name
closed_by
follow_up
parent_incident
reopened_by
u_hardware_swap
reassignment_count
u_end_of_lifecycle
assigned_to
sla_due
comments_and_work_notes
u_actual_resolution_time
u_affect_asset
u_replacement_asset
u_cause_code
escalation
upon_approval
correlation_id
u_location
made_sla
u_browsing_data_cleared
u_paging_response
child_incidents
task_effective_number
resolved_by
sys_updated_by
opened_by
user_input
sys_created_on
sys_domain
route_reason
calendar_stc
closed_at
u_error_code_account_alerts
u_windows_version
business_service
business_impact
rfc
time_worke


### Named Entity Recognition (NER)


Sizes: ['sm', 'md', 'lg']<br>
Your pipeline must be compatable with your current version of SpaCy.

Can download the following on Conda (base) environment:
`pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_[SIZE]-[VERSION]/en_core_web_md-[VERSION].tar.gz`

Alternatively, in Python:
`python -m spacy download en_core_web_sm`

In [66]:
from spacy.lang.en import English
nlp = English()

# Instantiate a Tokenizer with the default settings for English, including punctuation rules and exceptions.
tokenizer = nlp.tokenizer

In [67]:
# Assign SpaCy `en_core_web_[SIZE]` as `nlp`.
# python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_trf')
ner = nlp.get_pipe('ner')

In [68]:
# Example of token generation for the first body of text.
doc = nlp(df['short_description'][0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_) # Print: token, POS, syntactic dependency.

Pitney Bowes P2000 Series isn't able to connect with network
Pitney PROPN compound
Bowes PROPN compound
P2000 PROPN compound
Series PROPN nsubj
is AUX ROOT
n't PART neg
able ADJ acomp
to PART aux
connect VERB xcomp
with ADP prep
network NOUN pobj


In [69]:
# Create function to add an article's tokens to `doc_list`.
# Tokenize one time, then use that object for the subsequent accumulators.
# Returns None many times.
doc_list = []
def to_doc_list(text):
    doc_list.append(nlp(text))

In [70]:
# Takes time to generate tokens from each cell's fulltext.
df['short_description'].apply(to_doc_list)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
Name: short_description, dtype: object

In [71]:
# Assign `doc_list` to `doc_series` as a Series object.
doc_series = pd.Series(doc_list)
doc_series

0     (Pitney, Bowes, P2000, Series, is, n't, able, ...
1       (Can, not, connect, to, in, -, office, Desktop)
2             (Brittany, Tyler, -, Fixed, Income, Desk)
3     (Break, /, fix, Docking, station, not, providi...
4     (Baird, TrustDesk, Migration, to, OneDrive, -,...
5                      (Deposit, Edge, Account, Unlock)
6     (MFA, Authentication, Loop, -, Unable, to, acc...
7     (Beta, report, ran, out, of, paper, ., Needs, ...
8         (Laptop, not, charging, on, Docking, Station)
9               (Mac, not, connecting, to, Guest, wifi)
10    (RJ, Edgerly, needs, his, Outlook, inbox, rest...
11    (Hello, ., My, name, is, spelled, incorrectly,...
12    (Custom, example, :, Bryce, Townsend, tried, t...
dtype: object

In [72]:
for doc in doc_series:
    print(doc)
    filtered_string = ""
    for token in doc:
        if token.ent_type_ in ['PERSON', 'MONEY', 'CARDINAL', 'QUANTITY', 'PERCENT']:
            new_token = " <{}>".format(token.ent_type_)
        # elif token.pos_ in ['PROPN']:
        #     new_token = " <PROPN>"
        # elif token.pos_ in ['PROPN', 'NUM']:
        #     new_token = " <{}>".format(token.ent_type_)
        elif token.pos_ == "PUNCT":
            new_token = token.text
        else:
            new_token = " {}".format(token.text)
        filtered_string += new_token
    filtered_string = filtered_string[1:]
    print(filtered_string, '\n')

Pitney Bowes P2000 Series isn't able to connect with network
Pitney Bowes P2000 Series is n't able to connect with network 

Cannot connect to in-office Desktop
Can not connect to in- office Desktop 

Brittany Tyler - Fixed Income Desk
<PERSON> <PERSON>- Fixed Income Desk 

Break/fix Docking station not providing monitor display or network
Break / fix Docking station not providing monitor display or network 

Baird TrustDesk Migration to OneDrive - Error migrating 
Baird TrustDesk Migration to OneDrive- Error migrating 

Deposit Edge Account Unlock
Deposit Edge Account Unlock 

MFA Authentication Loop - Unable to access email
MFA Authentication Loop- Unable to access email 

Beta report ran out of paper. Needs a restart.
Beta report ran out of paper. Needs a restart. 

Laptop not charging on Docking Station
Laptop not charging on Docking Station 

Mac not connecting to Guest wifi
Mac not connecting to Guest wifi 

RJ Edgerly needs his Outlook inbox restored
<PERSON> <PERSON> needs his 


# Check NER for misses/gaps.



Dictionary accumulator for entities based on entity type


In [57]:
ent_dict = {}
def count_ent(doc):
    for ent in doc.ents:
        if ent.label_ not in ent_dict:
            ent_dict[ent.label_] = 1
        else:
            ent_dict[ent.label_]+=1
doc_series.apply(count_ent)
ent_dict

{'ORG': 2, 'PERSON': 4, 'PRODUCT': 1, 'CARDINAL': 1}

In [58]:
print(nlp.get_pipe("ner").labels)

('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')



#### PERSON.


In [59]:
person_dict = {}
def count_person(doc):
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            if ent.text not in person_dict:
                person_dict[ent.text]=1
            else:
                person_dict[ent.text]+=1
doc_series.apply(count_person)
sorted(person_dict.items(), key=lambda x: x[1], reverse=True)[0:10]

[('Brittany Tyler', 1),
 ('RJ Edgerly', 1),
 ('Veronica Fitzpatrick', 1),
 ('Bryce Townsend', 1)]


#### ORG (organization).


In [33]:
org_dict = {}
def count_org(doc):
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            if ent.text not in org_dict:
                org_dict[ent.text]=1
            else:
                org_dict[ent.text]+=1
doc_series.apply(count_org)
sorted(org_dict.items(), key=lambda x: x[1], reverse=True)[0:10]

[('Pitney Bowes P2000 Series', 1), ('DocuSign', 1)]


___

&nbsp;
#### Save out any of the above as a Series object


In [ ]:
ner_obj = sorted(org_dict.items(), key=lambda x: x[1], reverse=True) # specify which dict to save
ner_obj = pd.Series(ner_obj)
ner_obj
# ner_obj.to_csv(output_dir+'ner_obj.csv', sep=';', encoding='utf-8-sig')


***
